The following can only be run once per kernel restart

In [1]:
import multiprocessing as mp
mp.set_start_method('fork')

In [2]:
import pandas as pd
import numpy as np
import logging
from backtesting import Backtest
from backtesting.lib import resample_apply, plot_heatmaps, TRADES_AGG
logging.basicConfig(level=logging.WARNING, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

from strategies import LONG_SHORT_Underwater_w_decay_and_deleverage


## Create a helper function to keep your repo clean
Everytime you run a backtest it saves a file in your root. now it will save it in a folder called `html_files`

In [3]:
# This will be handy when we want to plot so it doesn't store all the html files in the root
def create_filename(stats_df):
    params = stats_df._strategy.__dict__['_params']
    filename = 'html_files/strat'
    # unpack the best params and add them to a filename
    for key, value in params.items():
        # keep the first 5 letters from each key
        key = key[:5]
        # just keep the first 4 digits from each value
        value= str(value)[:4]
        filename += f'_{key}_{value}_'
    filename += '.html'
    return filename
# Check if the folder html_files exists, if not create it
import os
if not os.path.exists('html_files'):
    os.makedirs('html_files')

# Strategy combining Long and Short signals

Our signals are 1 at 00:01 every day and -1 at 12:00 every day. If we are not in a trade, then whichever signal comes first it will work it. 
@hoang, if you can read in the lstm buy signals as a 1 and open sell signals as a -1 then I think this will all work right away. We will have to think about duplicated signals but I don't think that is a problem. 

# Add in our signals
Run the strategies long and short based on our own AI buy and sell signals. Use it as an opportunity to improve the results of our existing AI models.
#### Read in a dataframe of our signals
prep the dataframe so it has the `Close` price, the `signal` column. 
For this we can create an `entries` column and an `exits` column.

| trigger (from CSV)  | entries          | exits            | signal |
| --------            | --------         | --------         | --     |
| 1 day passed        | None             | 1 day passed     | 0      |
| lstm_open-long      | lstm_open-long   | None             | 1      |
| lstm_open-short     | lstm_open-short  | None             | -1     |
| NA                  | None             | None             | 0      |
| stop loss           | None             | stop loss        | 0      |
| take profit         | None             | take profit      | 0      |

                      

We can then tweak our code above to read from the `entries` column instead of the `signal` column. We will be ignoring the LSTM exits for now because the backtest strategy takes over once we are in a trade. Perhaps this is something we will change in the future but for now we can keep it as is.


# Import the trades dataframe and manipulate it.

## Step 1 - Read in the CSV files

In [4]:
from typing import List
lstm_csv_files = ["test_data/2dc0f4b2_minutely.csv",
"test_data/3dca0a12_minutely.csv",
"test_data/9295144c_minutely.csv",
"test_data/b2d47ab1_minutely.csv",
"test_data/b9962cd1_minutely.csv"]

def _read_lstm_files() -> List:
  lstm_dfs = []

  for file in lstm_csv_files:
    lstm_dfs.append(pd.read_csv(file))

  return lstm_dfs
    

## Define constants and mappings

In [5]:
from dataclasses import dataclass
from typing import Optional
from collections import defaultdict

@dataclass
class BackTestColumnValues:
  entries : Optional[str]
  exits   : Optional[str]
  signal  : int

LSTM_TRIGGER_MAP = defaultdict(
  lambda            : BackTestColumnValues(None             , None            , 0 ), 
{
  "1 day passed"    : BackTestColumnValues(None             , "1 day passed"  , 0 ),
  "lstm_open-long"  : BackTestColumnValues("lstm_open-long" , None            , 1 ),
  "lstm_open-short" : BackTestColumnValues("lstm_open-short", None            , -1),  
  "N/A"             : BackTestColumnValues(None             , None            , 0 ),
  "stop loss"       : BackTestColumnValues(None             , "stop loss"     , 0 ),
  "take profit"     : BackTestColumnValues(None             , "take profile"  , 0 ),
})



## Define the transformation 

In [6]:
def _map_lstm_trigger_to_signal(trigger: str) -> Optional[int]:
  return LSTM_TRIGGER_MAP[trigger].signal


def _map_lstm_trigger_to_entries(trigger: str) -> Optional[str]:
  return LSTM_TRIGGER_MAP[trigger].entries


def _map_lstm_trigger_to_exits(trigger: str) -> Optional[str]:
  return LSTM_TRIGGER_MAP[trigger].exits


def _transform_lstm_dfs(lstm_dfs: List):
  for lstm_df in lstm_dfs:    
    lstm_df.rename(columns={'price': 'Close'}, inplace=True)

    lstm_df['trigger']  = lstm_df['trigger'].astype(str)
    lstm_df['entries']  = lstm_df['trigger'].apply(_map_lstm_trigger_to_entries)
    lstm_df['exits']    = lstm_df['trigger'].apply(_map_lstm_trigger_to_exits)
    lstm_df['signal']   = lstm_df['trigger'].apply(_map_lstm_trigger_to_signal)    
    lstm_df['time']     = pd.to_datetime(lstm_df['time'], unit='s').dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

## The starting point of backtesting LSTM CSV files

In [7]:
lstm_dfs = _read_lstm_files()
_transform_lstm_dfs(lstm_dfs)

Need to make a slight mod to our filename func

In [8]:
# This will be handy when we want to plot so it doesn't store all the html files in the root
def create_filename(stats_df):
    params = stats_df._strategy.__dict__['_params']
    filename = 'html_files/strat'
    # unpack the best params and add them to a filename
    for key, value in params.items():
        # keep the first 5 letters from each key
        key = key[:5]
        # just keep the first 4 digits from each value
        value= str(value)[:4]
        filename += f'_{key}_{value}_'
    # filename += '.html'
    return filename

In [9]:
test_df = lstm_dfs[0]
# Rename open, high, low, close to Open, High, Low, Close
test_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
test_df.sort_index(inplace=True)

In [10]:
stats_list = []
for i in range(len(lstm_dfs)):
    test_df = lstm_dfs[i]
    test_df.index = pd.to_datetime(test_df['time']) 
    test_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    start = "2022-01-09"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
    end = "2023-05-10"  # It will always close any open trades at the end of the backtest
    bt = Backtest(
        test_df.loc[start:end],
        LONG_SHORT_Underwater_w_decay_and_deleverage,
        cash=100_000_000,
        exclusive_orders=False,
        trade_on_close=True,
        margin=1,
    )
    stats = bt.run()
    stats_list.append(stats)
    filename = create_filename(stats)
    print(stats)
    bt.plot(resample=False, filename=f'{filename}_lstm_{i}.html')
    

/var/folders/dt/m1bsm13n187d7m0ssssp79ww0000gn/T/ipykernel_52452/1982906418.py:8: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(


In [ ]:
# Concatenate all the stats into one dataframe. They are all currently series, I want them all to use the same index
stats_df = pd.concat(stats_list, axis=1)
stats_df

,0,1,2,3,4
Start,2022-01-09 00:00:00+00:00,2022-01-09 00:00:00+00:00,2022-01-09 00:00:00+00:00,2022-01-09 00:00:00+00:00,2022-01-09 00:00:00+00:00
End,2023-05-10 23:06:00+00:00,2023-05-10 22:44:00+00:00,2023-05-10 23:32:00+00:00,2023-05-10 21:50:00+00:00,2023-05-10 23:32:00+00:00
Duration,486 days 23:06:00,486 days 22:44:00,486 days 23:32:00,486 days 21:50:00,486 days 23:32:00
Exposure Time [%],95.696116,93.288553,95.407364,94.643664,93.29928
Equity Final [$],74432559.17,131637667.06,85330629.52,76537601.38,128700632.77
Equity Peak [$],101265872.33,166844980.36,109351025.02,104526239.98,165201803.67
Return [%],-25.567441,31.637667,-14.66937,-23.462399,28.700633
Buy & Hold Return [%],-33.63278,-33.882012,-33.840753,-33.993794,-33.840753
Return (Ann.) [%],-19.852857,22.877919,-11.209995,-18.159976,20.817337
Volatility (Ann.) [%],20.445416,29.240466,22.672516,20.038167,30.94416


Looks like 1 and 4 are best.

In [ ]:
# Save the dataframe to a csv after appending `lstm` as a prefix to each column name
stats_df.columns = [f'lstm_{col}' for col in stats_df.columns]
stats_df.to_csv('lstm_stats.csv')

# Run the backtest on the lstm entries from a single dataframe
Based on looking at the stats 1 and 4 had the best results and were fairly similar. I'm going to run this on 1. You can choose to optimize it on any of them.

In [ ]:
test_df = lstm_dfs[1] # <-- Change this to the index of the dataframe 0,1,2,3,4 you want to plot
test_df.index = pd.to_datetime(test_df['time']) 
test_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
test_df.sort_index(inplace=True)

In [ ]:
start = "2021-01-09"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2022-03-10"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    test_df.loc[start:end],
    LONG_SHORT_Underwater_w_decay_and_deleverage,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=1,
)
stats = bt.run()
filename = create_filename(stats)
print(stats)
bt.plot(resample=False, filename=filename)

Start                     2021-01-09 00:01...
End                       2022-03-10 23:59...
Duration                    425 days 23:58:00
Exposure Time [%]                   95.183384
Equity Final [$]                 104000680.31
Equity Peak [$]                  111362287.34
Return [%]                            4.00068
Buy & Hold Return [%]               -3.068876
Return (Ann.) [%]                    3.418141
Volatility (Ann.) [%]               26.764548
Sharpe Ratio                         0.127712
Sortino Ratio                        0.191836
Calmar Ratio                         0.144507
Max. Drawdown [%]                  -23.653836
Avg. Drawdown [%]                   -1.164814
Max. Drawdown Duration      212 days 12:25:00
Avg. Drawdown Duration        4 days 15:15:00
# Trades                                  765
Win Rate [%]                         26.27451
Best Trade [%]                      20.022618
Worst Trade [%]                    -77.879826
Avg. Trade [%]                    

GridPlot(id='p3039', ...)

# Optimize the parameters

In [ ]:
stats, heatmap = bt.optimize(
    initial_position_size = np.arange(0.3, 0.6, 0.1).tolist(),
    percent_invested_threshold = np.arange(0.3, 0.8, 0.1).tolist(),
    atr_length = np.arange(12,20,2).tolist(), # 14 days
    atr_multiplier = np.arange(0.3, 0.5, 0.1).tolist(),
    add_size = np.arange(0.05,0.20, 0.05).tolist(),
    delay_period = np.arange(250,750,250).tolist(),
    delta_time = np.arange(500,1000,100).tolist(),
    upper_bound_profit_target = np.arange(0.005, 0.04, 0.005).tolist(),
    lower_bound_loss_threshold = np.arange(-0.10, 0.00, 0.02).tolist(),
    # take_profit_loss_reduction = np.arange(-0.15, -0.05, 0.05).tolist(), # This is the amount that the take profit is reduced by if the position is highly leveraged and we wish to trim
    deleverage_pct = np.arange(0.25, 0.75, 0.25).tolist(), # This is the amount that the position is reduced by if the position is highly leveraged and we wish to trim
    max_loss_threshold = np.arange(-0.15, -0.05, 0.05).tolist(),
    max_hold_length = np.arange(60*8, 60*48, 60*4).tolist(), # 8 hours to 48 hours
    maximize='Equity Final [$]', 
    # maximize='Max. Drawdown [%]',# this can be any of the column names from the stats table the output of the backtest
    # maximize='Win Rate [%]',
    max_tries=200,
    random_state=0,
    
    return_heatmap=True)
best_params = stats._strategy.__dict__["_params"] # This will print out all the parameters used for the best backtest
print(f'Best Parameters: {best_params}')
heatmap.sort_values(ascending=False).iloc[:5] # print the top 5 parameter sets

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Best Parameters: {'initial_position_size': 0.4, 'percent_invested_threshold': 0.5, 'atr_length': 12, 'atr_multiplier': 0.4, 'add_size': 0.05, 'delay_period': 250, 'delta_time': 900, 'upper_bound_profit_target': 0.02, 'lower_bound_loss_threshold': -0.1, 'deleverage_pct': 0.5, 'max_loss_threshold': -0.09999999999999999, 'max_hold_length': 1680}


initial_position_size  percent_invested_threshold  atr_length  atr_multiplier  add_size  delay_period  delta_time  upper_bound_profit_target  lower_bound_loss_threshold  deleverage_pct  max_loss_threshold  max_hold_length
0.4                    0.5                         12          0.4             0.05      250           900         0.020                      -0.10                       0.50            -0.10               1680               1.354023e+08
                       0.4                         12          0.4             0.10      500           600         0.025                      -0.06                       0.50            -0.10               1680               1.298580e+08
0.5                    0.6                         18          0.3             0.05      250           600         0.030                      -0.06                       0.25            -0.10               2400               1.293923e+08
0.3                    0.6                         18          0

In [ ]:
stats

Start                     2021-01-09 00:01...
End                       2022-03-10 23:59...
Duration                    425 days 23:58:00
Exposure Time [%]                   80.743318
Equity Final [$]                 135402323.09
Equity Peak [$]                  149505592.19
Return [%]                          35.402323
Buy & Hold Return [%]               -3.068876
Return (Ann.) [%]                   29.651709
Volatility (Ann.) [%]               39.092791
Sharpe Ratio                         0.758496
Sortino Ratio                        1.479336
Calmar Ratio                          1.19009
Max. Drawdown [%]                  -24.915526
Avg. Drawdown [%]                   -0.503028
Max. Drawdown Duration      113 days 10:42:00
Avg. Drawdown Duration        0 days 23:10:00
# Trades                                 1540
Win Rate [%]                        58.376623
Best Trade [%]                      13.150986
Worst Trade [%]                    -26.565973
Avg. Trade [%]                    

In [ ]:
# Plot the heatmaps
plot_heatmaps(heatmap, agg='mean')


GridPlot(id='p8159', ...)

Run the best strategy

In [ ]:
bt.run(**best_params)
bt.plot(resample='2h', filename=f'{filename}_optimized.html')

GridPlot(id='p13557', ...)

In [ ]:
best_params

{'initial_position_size': 0.4,
 'percent_invested_threshold': 0.5,
 'atr_length': 12,
 'atr_multiplier': 0.4,
 'add_size': 0.05,
 'delay_period': 250,
 'delta_time': 900,
 'upper_bound_profit_target': 0.02,
 'lower_bound_loss_threshold': -0.1,
 'deleverage_pct': 0.5,
 'max_loss_threshold': -0.09999999999999999,
 'max_hold_length': 1680}

Run the full backtest on the entire period but use the params that were optimized on 2021 data

In [ ]:
# run it on the full period
start = "2021-01-09"  # Note the strategy requires a warmup period for the ATR to calculate first trades begin after 14 days
end = "2022-03-10"  # It will always close any open trades at the end of the backtest
bt = Backtest(
    test_df, #.loc[start:end],
    LONG_SHORT_Underwater_w_decay_and_deleverage,
    cash=100_000_000,
    exclusive_orders=False,
    trade_on_close=True,
    margin=1,
)
stats = bt.run(**best_params)
filename = create_filename(stats)
print(stats)
bt.plot(resample='2h', filename=filename)

Start                     2021-01-09 00:01...
End                       2023-05-10 23:06...
Duration                    851 days 23:05:00
Exposure Time [%]                   86.873118
Equity Final [$]                 104256409.31
Equity Peak [$]                  187790963.81
Return [%]                           4.256409
Buy & Hold Return [%]              -31.967011
Return (Ann.) [%]                    1.801761
Volatility (Ann.) [%]               37.270378
Sharpe Ratio                         0.048343
Sortino Ratio                        0.071419
Calmar Ratio                         0.036554
Max. Drawdown [%]                  -49.290922
Avg. Drawdown [%]                   -0.755592
Max. Drawdown Duration      325 days 17:40:00
Avg. Drawdown Duration        1 days 23:24:00
# Trades                                 2098
Win Rate [%]                        53.813155
Best Trade [%]                      10.776911
Worst Trade [%]                    -32.653414
Avg. Trade [%]                    

GridPlot(id='p61565', ...)

# Test on different periods of time Possibly set up a cross validation
Nice youtube video here https://www.youtube.com/watch?v=9m987swadQU&t=2154s&ab_channel=ChadThackray